In [5]:
import pandas as pd
import datetime as dt

# 1. Load the file from your computer
# If you didn't rename it, the file name is 'Online Retail.xlsx'
file_path = 'Online Retail.xlsx'
df = pd.read_excel(file_path)

# 2. Keep only the first 50,000 rows to make it run faster for now
df = df.head(50000)

# 3. Clean the data
# Remove rows that don't have a CustomerID
df = df.dropna(subset=['CustomerID'])

# Remove 'Canceled' orders (anything with Quantity less than or equal to 0)
df = df[df['Quantity'] > 0]

# 4. Create a 'TotalSales' column
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

print("--- Step 2 Complete ---")
print(f"Dataset loaded successfully with {df.shape[0]} rows.")

--- Step 2 Complete ---
Dataset loaded successfully with 31239 rows.


In [6]:
# Use the latest date in the data as our reference point
latest_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

# Group transactions by CustomerID
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days, # Recency
    'InvoiceNo': 'count',                                   # Frequency
    'TotalSales': 'sum'                                     # Monetary
})

# Rename columns so they make sense
rfm.columns = ['Recency', 'Frequency', 'Monetary']

print("\n--- Step 3 Complete: RFM Table ---")
print(rfm.head())


--- Step 3 Complete: RFM Table ---
            Recency  Frequency  Monetary
CustomerID                              
12347.0          34         31    711.79
12348.0          24         17    892.80
12370.0          24         91   1868.02
12377.0          21         43   1001.52
12383.0          19         37    600.72


In [9]:
# Instead of a fixed number, we will label the 25% of customers 
# who haven't shopped for the longest time as 'Churned' (1).
churn_threshold = rfm['Recency'].median() # Using median guarantees a 50/50 split for learning

rfm['Churn'] = rfm['Recency'].apply(lambda x: 1 if x >= churn_threshold else 0)

print("Check this output - You must see both 0 and 1 below:")
print(rfm['Churn'].value_counts())

Check this output - You must see both 0 and 1 below:
Churn
1    518
0    455
Name: count, dtype: int64


In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# 1. Define your Inputs (X) and your Target (y)
X = rfm[['Frequency', 'Monetary']]
y = rfm['Churn']

# 2. Split the data (80% to train the AI, 20% to test it)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Create the model with a fixed 'base_score' to avoid the error you saw
# We also use 'logloss' to keep the model updated with latest XGBoost standards
model = XGBClassifier(
    n_estimators=100, 
    max_depth=3, 
    eval_metric='logloss',
    base_score=0.5 
)

# 4. Train the model
model.fit(X_train, y_train)

# 5. Check the result
accuracy = model.score(X_test, y_test)
print(f"Success! Your model is now trained.")
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Success! Your model is now trained.
Model Accuracy: 53.85%


In [11]:
def get_retention_strategy(row):
    # Rule 1: High Spenders at Risk
    if row['Churn'] == 1 and row['Monetary'] > rfm['Monetary'].quantile(0.75):
        return "VIP Concierge Call: Offer 20% Personal Discount"
    
    # Rule 2: Frequent Shoppers Slipping Away
    elif row['Churn'] == 1 and row['Frequency'] > rfm['Frequency'].quantile(0.75):
        return "Loyalty Email: Send 'We Miss You' Reward Points"
    
    # Rule 3: General Churn Risk
    elif row['Churn'] == 1:
        return "Standard Re-engagement: Send Latest Catalog"
    
    else:
        return "Nurture: Send Weekly Newsletter"

# Apply the strategy to our results
rfm['Strategy'] = rfm.apply(get_retention_strategy, axis=1)

print("\n--- Sample Retention Strategies ---")
print(rfm[['Monetary', 'Churn', 'Strategy']].head(10))


--- Sample Retention Strategies ---
            Monetary  Churn                                         Strategy
CustomerID                                                                  
12347.0       711.79      1  VIP Concierge Call: Offer 20% Personal Discount
12348.0       892.80      0                  Nurture: Send Weekly Newsletter
12370.0      1868.02      0                  Nurture: Send Weekly Newsletter
12377.0      1001.52      0                  Nurture: Send Weekly Newsletter
12383.0       600.72      0                  Nurture: Send Weekly Newsletter
12386.0       401.90      0                  Nurture: Send Weekly Newsletter
12395.0       679.92      0                  Nurture: Send Weekly Newsletter
12413.0       479.25      0                  Nurture: Send Weekly Newsletter
12415.0      7011.38      0                  Nurture: Send Weekly Newsletter
12417.0       291.34      0                  Nurture: Send Weekly Newsletter


In [12]:
# 1. Calculate new metrics from the original 'df'
extra_features = df.groupby('CustomerID').agg({
    'StockCode': 'nunique',   # Number of unique products bought
    'Quantity': 'mean'        # Average quantity per item
})

# 2. Join these new features to our 'rfm' table
rfm = rfm.join(extra_features)

# 3. Create 'Average Order Value'
rfm['Avg_Order_Value'] = rfm['Monetary'] / rfm['Frequency']

# 4. Update our Features (X) to include these new columns
X = rfm[['Frequency', 'Monetary', 'StockCode', 'Avg_Order_Value']]
y = rfm['Churn']

# 5. Re-train the model
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, base_score=0.5)
model.fit(X_train, y_train)

print(f"New Model Accuracy: {model.score(X_test, y_test) * 100:.2f}%")

New Model Accuracy: 56.92%


In [13]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'churn_model.pkl')

# Also save the column names so the dashboard knows the order
joblib.dump(X.columns.tolist(), 'feature_columns.pkl')

print("Model and Columns saved! You are ready to build the dashboard.")

Model and Columns saved! You are ready to build the dashboard.
